In [1]:
import os
import sys

# 1. Clone the AutoVision Repo
REPO_URL = "https://github.com/Kaankocc/autovision-cityscapes.git"
REPO_NAME = "autovision-cityscapes"
if not os.path.exists(REPO_NAME):
    !git clone {REPO_URL}

# 2. Add to Python Path
PROJECT_PATH = os.path.abspath(REPO_NAME)
SRC_PATH = os.path.join(PROJECT_PATH, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

# 3. Fix Imports (The "src." remover)
for filename in ['utils.py', 'dataset.py', 'model.py']:
    path = os.path.join(SRC_PATH, filename)
    with open(path, 'r') as f:
        content = f.read()
    if 'from src.' in content:
        with open(path, 'w') as f:
            f.write(content.replace('from src.', 'from '))
        print(f"✅ Fixed imports in {filename}")

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Fine-Tuning Environment Ready on {device}")

Cloning into 'autovision-cityscapes'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 4), reused 20 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 9.25 MiB | 29.04 MiB/s, done.
Resolving deltas: 100% (4/4), done.
✅ Fixed imports in utils.py
🚀 Fine-Tuning Environment Ready on cuda


In [2]:
from utils import bake_masks
import shutil

# 1. Define Paths
RAW_DATA = "/kaggle/input/cityscapes-image-pairs/cityscapes_data/cityscapes_data"
PROC_DATA = "/kaggle/working/processed_masks"

# 2. Bake Masks (if not already done)
if not os.path.exists(PROC_DATA):
    print(f"🔥 Baking masks from {RAW_DATA}...")
    bake_masks(RAW_DATA, PROC_DATA)
    print(f"✅ Masks baked at {PROC_DATA}")
else:
    print("📂 Masks already exist. Skipping bake.")

🔥 Baking masks from /kaggle/input/cityscapes-image-pairs/cityscapes_data/cityscapes_data...
🔥 Baking 2975 masks for 'train' split...


100%|██████████| 2975/2975 [02:15<00:00, 21.98it/s]


🔥 Baking 500 masks for 'val' split...


100%|██████████| 500/500 [00:22<00:00, 22.23it/s]

✅ Masks baked at /kaggle/working/processed_masks


In [ ]:
from model import UNet
from dataset import CityscapesKaggleDataset
from torch.utils.data import DataLoader
import os

BATCH_SIZE = 4        
NUM_WORKERS = 2       

PRETRAINED_PATH = "/kaggle/input/cityscapes-512-best-pth/pytorch/default/1/cityscapes_512_best.pth" 

# 2. Initialize the Model
model = UNet(n_channels=3, n_classes=7).to(device)

# 3. Load the Weights
if os.path.exists(PRETRAINED_PATH):
    print(f"📂 Loading weights from: {PRETRAINED_PATH}")
    model.load_state_dict(torch.load(PRETRAINED_PATH, map_location=device))
    print("🧠 Foundation weights loaded successfully.")
else:
    print(f"❌ ERROR: Could not find file at {PRETRAINED_PATH}")
    print("Please check the path in the sidebar's 'Input' section.")

# 4. Prepare Data Loaders (With Safe Settings)
print("📊 Initializing DataLoaders with Safe Mode...")
train_ds = CityscapesKaggleDataset(root_dir=RAW_DATA, split='train', target_size=(512, 512))

train_loader = DataLoader(
    train_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=NUM_WORKERS, 
    pin_memory=True         
)

print(f"✅ Ready to Train: {len(train_ds)} images | Batch Size: {BATCH_SIZE} | Workers: {NUM_WORKERS}")

📂 Loading weights from: /kaggle/input/cityscapes-512-best-pth/pytorch/default/1/cityscapes_512_best.pth
🧠 Foundation weights loaded successfully.
📊 Initializing DataLoaders with Safe Mode...
✅ Ready to Train: 2975 images | Batch Size: 4 | Workers: 2


In [ ]:
import torch.nn as nn
import torch.optim as optim

# 1. Class Weights for Fine-Tuning
fine_tune_weights = torch.tensor([1.0, 1.0, 20.0, 2.0, 1.0, 20.0, 1.0]).to(device)

# 2. Loss Function
criterion = nn.CrossEntropyLoss(weight=fine_tune_weights)

# 3. Optimizer with Low Learning Rate
optimizer = optim.Adam(model.parameters(), lr=1e-5)

print("✅ Fine-Tuning Configuration Set:")
print("   - LR: 1e-5")
print("   - Human/Object Weight: 20.0")

✅ Fine-Tuning Configuration Set:
   - LR: 1e-5
   - Human/Object Weight: 20.0


In [5]:
from tqdm import tqdm

EPOCHS = 5
best_loss = float('inf')

print(f"📡 Starting Surgical Fine-Tuning for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for batch_idx, (images, masks) in pbar:
        images, masks = images.to(device), masks.to(device)
        
        # Forward
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
    
    avg_loss = running_loss / len(train_loader)
    print(f"🏁 Epoch {epoch+1} Done | Avg Loss: {avg_loss:.4f}")
    
    # Save Checkpoint
    torch.save(model.state_dict(), "cityscapes_finetuned.pth")
    print("💾 Model saved.")

print("\n✨ Fine-Tuning Complete! Download 'cityscapes_finetuned.pth'.")

📡 Starting Surgical Fine-Tuning for 5 epochs...


Epoch 1/5: 100%|██████████| 744/744 [13:46<00:00,  1.11s/it, loss=0.1795]


🏁 Epoch 1 Done | Avg Loss: 0.1676
💾 Model saved.


Epoch 2/5: 100%|██████████| 744/744 [14:03<00:00,  1.13s/it, loss=0.1327]


🏁 Epoch 2 Done | Avg Loss: 0.1601
💾 Model saved.


...

🏁 Epoch 3 Done | Avg Loss: 0.1547
💾 Model saved.


Epoch 4/5: 100%|██████████| 744/744 [14:06<00:00,  1.14s/it, loss=0.1809]


🏁 Epoch 4 Done | Avg Loss: 0.1515
💾 Model saved.


Epoch 5/5: 100%|██████████| 744/744 [14:05<00:00,  1.14s/it, loss=0.1770]


🏁 Epoch 5 Done | Avg Loss: 0.1483
💾 Model saved.

✨ Fine-Tuning Complete! Download 'cityscapes_finetuned.pth'.
